# GraphLab Create v Common Crawl 2012 WebGraph
## or, How I Learned to Stop Worrying about 128B edges and Love the PageRank

Here at Dato we highly evaluate openness and transparency. Look at the [Dato Gallery](https://dato.com/learn/gallery/ "Dato Gallery"), filled with notebooks you can download and execute on your own machine, to see that that GraphLab Create can really do what we say it does. So when it came to benchmarks, we said, why not upload a notebook about it?

This notebook will describe our **PageRank benchmark**. The dataset for this benchmark is the web itself - compiled by good people from [commoncrawl.org](http://commoncrawl.org/) in 2012. You can [download the dataset from here](http://webdatacommons.org/hyperlinkgraph/2012-08/download.html#toc0). We will run the **PageRank** algorithm over a network of **3.5 billion nodes** and **128 billion links**. Each node represents a web page, and each link - a hyperlink between two pages.

Running this benchmark will prove you how powerful and robust GraphLab is - not many systems with graphs of this size. However, unlike other notebooks in the gallery, we **don't recommend running this notebook on your laptop!**. Instead, this notebook will describe how to run this benchmark on an EC2 instance in the Amazon Web Services (AWS) cloud.

We'll be using an **r3.8xlarge** EC2 instance,. That's a strong machine,
with 32 cores, 244 Gigabytes of RAM, and 2 SSD drives, each sized 320 GBs.
If you can access a physical machine of this calibre, expect similar results.

For now, read our guide for **launching and setting-up an EC2 instance** for this benchmark. This guide will describe how to:

1. Launch an EC2 instance,
2. Install GraphLab Create and Jupyter (formerly *IPython Notebook*) on it,
3. Download this benchmark notebook to the EC2 machine,
4. Connect to your Jupyter instance running on the EC2 machine and run the benchmark.

# The Data

The [dataset's webpage says](http://webdatacommons.org/hyperlinkgraph/2012-08/download.html#toc0):
 
    Downloading the page graph: The page graph (arc and indes files) are, due to their size split into in small files of around 500 MB. These files can be downloaded using ```wget -i http://webdatacommons.org/hyperlinkgraph/2012-08/data/index.list.txt``` for the index files and respectively ```wget -i http://webdatacommons.org/hyperlinkgraph/2012-08/data/arc.list.txt``` for the arc files.

Examining http://webdatacommons.org/hyperlinkgraph/2012-08/data/arc.list.txt , we will find a list of URLs pointing at .gz files.

```
http://data.dws.informatik.uni-mannheim.de/hyperlinkgraph/2012-08/network/part-r-00000.gz
http://data.dws.informatik.uni-mannheim.de/hyperlinkgraph/2012-08/network/part-r-00001.gz
...
http://data.dws.informatik.uni-mannheim.de/hyperlinkgraph/2012-08/network/part-r-00696.gz
```

As CommonCrawl's own documentation says, each of these gzip files weights ~500 MBs. Here is the `head` of the first file (`part-r-00000.gz`):

```
0	739935047
1	741742773
2	741745070
```

This is a very common format for storing graph edges - id1-TAB-id2.
While GraphLab could handle loading such a graph, we saved you the the trouble of downloading the files to your EC2 instance and uploaded it to an open S3 bucket. To access it you only need AWS credentials.

The SGraph created by loading the data is stored in binary form in this bucket; that way, it weights around 218 GBs, less than the gzipped edges.
Also, since this data is stored on Amazon's S3, accessing it from your EC2 instance should be faster than downloading the raw data from CommonCrawl's servers.

The bucket is located at [s3://dato-datasets-oregon/webgraphs/sgraph/common_crawl_2012_sgraph/](s3://dato-datasets-oregon/webgraphs/sgraph/common_crawl_2012_sgraph/) .

```bash
# Password protect Jupyter
$ jupyter notebook --generate-config
$ python -c "from notebook.auth import passwd; password = passwd(); open('/home/ubuntu/.jupyter/jupyter_notebook_config.py', 'a').write('c.NotebookApp.password = u\'%s\'' % (password))"
```